In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)
# Change this based on the drive being used
# my_voice_data = '/content/drive/MyDrive/Project_B/Voice samples study'
my_voice_data = '/content/drive/MyDrive/Study_materials/Voice samples study'

In [ ]:
!apt-get install libsox-fmt-all
!apt-get install sox
!pip install sox

In [ ]:
import sox

In [ ]:
!sox /content/drive/MyDrive/Study_materials/Voice\ samples\ study/Patient-127.wav -c 1 pat_127_sox.wav
!sox /content/pat_127_sox.wav -r 16000 /content/pat_127_sox_mono.wav
!rm pat_127_sox.wav


!sox /content/drive/MyDrive/Study_materials/Voice\ samples\ study/Patient\ 01.wav -c 1 pat_01_sox.wav
!sox /content/pat_01_sox.wav -r 16000 /content/pat_01_sox_mono.wav
!rm pat_01_sox.wav

!sox /content/drive/MyDrive/Study_materials/Voice\ samples\ study/Patient\ 97.wav -c 1 pat_97_sox.wav
!sox /content/pat_97_sox.wav -r 16000 /content/pat_97_sox_mono.wav
!rm pat_97_sox.wav

!sox /content/drive/MyDrive/Study_materials/Voice\ samples\ study/Patient\ 151.wav -c 1 pat_151_sox.wav
!sox /content/pat_151_sox.wav -r 16000 /content/pat_151_sox_mono.wav
!rm pat_151_sox.wav

In [ ]:
!sox --i /content/pat_127_sox_mono.wav
!sox --i /content/pat_01_sox_mono.wav
!sox --i /content/pat_97_sox_mono.wav
!sox --i /content/pat_151_sox_mono.wav

In [ ]:
## Imports

import os
import glob
import numpy as np
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
from IPython.display import Audio

print(torch.__version__)
print(torchaudio.__version__)

## Use cuda if available
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

In [ ]:
# Define constants
wanted_sample_rate = 16000

audio_labels = ["normal pitch E","low pitch E","high pitch E",
                "normal pitch Ah","low pitch Ah","high pitch Ah",
                "normal pitch Ooh","low pitch Ooh","high pitch Ooh",
                "normal pitch Mmmmm"]

def plot_waveform(waveform, sample_rate):
    waveform = waveform.numpy()
    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
    figure.suptitle("waveform")
    plt.show(block=False)

Load the data using torchaudio
(probably not going to work, need to use SoX)

In [ ]:
# ### Load Data ##

# # First we count number of .wav files
# counter = 0
# for filename in os.listdir(my_voice_data):
#     if filename.endswith("wav"):
#       counter +=1
# print(f'We have {counter} .wav files')

# # We bucket sort all patients into an array of mono-channel, 16k S/R audiotorch tensors.
# patients_audio_list = [None] * (counter + 10) # We add +2 because there's a mis-lable in filenames
# for filename in os.listdir(my_voice_data):
#     if filename.endswith("wav"):
#       # Extract patient's ID from filename
#         _, numberwav = filename.split(" ")
#         number = numberwav.split(".")
#         patient_id = number[0]
#         waveform, sample_rate = torchaudio.load(my_voice_data +'/'+ filename)
#         transform = T.Resample(sample_rate, wanted_sample_rate)
#         waveform = transform(waveform)
#         waveform_mono = torch.mean(waveform, dim=0).unsqueeze(0)
#         # if sample_rate != wanted_sample_rate:
#         #   waveform = torchaudio.functional.resample(waveform, sample_rate, wanted_sample_rate)
#         patients_audio_list[int(patient_id)] = (int(patient_id),waveform_mono,wanted_sample_rate)


Google's VAD import, defs and methods:

In [ ]:
!pip install webrtcvad
import webrtcvad

In [ ]:
import collections
import contextlib
import sys
import wave



def read_wave(path):
    """Reads a .wav file.

    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.

    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)



class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.

    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.

    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.

    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.

    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.

    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.

    Arguments:

    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).

    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])


google's VAD usage method:

In [ ]:
def webrtcvad_usage(aggressive=0, audio_path=None,frame_duration=30,padding_duration=300,min_chunk_length = 0.5):
    if os.path.isfile('/content/chunk-00.wav'):
      print("Clearing previous .wav files")
      !rm chunk-??.wav
      !rm chunk-???.wav
    vad = webrtcvad.Vad(aggressive)
    audio, sample_rate  = read_wave(audio_path)
    frames = frame_generator(frame_duration, audio, sample_rate)
    frames = list(frames)
    segments = vad_collector(sample_rate, frame_duration, padding_duration, vad, frames)



  #  sample_rate - The audio sample rate, in Hz.
  #   frame_duration_ms - The frame duration in milliseconds.
  #   padding_duration_ms - The amount to pad the window, in milliseconds.
  #   vad - An instance of webrtcvad.Vad.
  #   frames - a source of audio frames (sequence or generator).
    i = 0
    for j,segment in enumerate(segments):
      path = 'chunk-%002d.wav' % (i,)
      # print(' Writing %s' % (path,))
      len_sec = len(segment)/(sample_rate*2)
      # if( i < 10):
      if((len_sec >= min_chunk_length) ):
        write_wave(path, segment, sample_rate)
        # print(f'\n phone Chunk {i} length is {len_sec} seconds')
        i = i+1
      # else:
        # print(f'\n too short! Chunk {i} length is {len_sec} seconds')
    return segments

In [ ]:
!pip install dtw-python

Roni's DTW code:

In [ ]:
_author_ = "Roni Chernyak"

import librosa
import numpy as np
import os
from scipy.spatial.distance import euclidean
from dtw import dtw
from scipy import stats

TEST_PATH = "/content/chunk-09.wav"
GT_PATH = "/content/mmm_GTT.wav"


# dynamic programming implementation
def DTW(src, trgt):
    dtw = np.full((src.shape[1]+1, trgt.shape[1]+1), np.inf, dtype=np.float64)
    dtw[0,0] = 0
    for i in range(1, src.shape[1]+1):
        for j in range(1, trgt.shape[1]+1):
            c = euclidean(src[:,i-1], trgt[:,j-1]) # Euclidean distance
            #c = np.abs(src[i] - trgt[j])
            dtw[i][j] = c + np.min([dtw[i - 1][j],
                                    dtw[i][j - 1],
                                    dtw[i - 1][j - 1]])
    # get the dtw distance
    return dtw[-1, -1]

def libroa_dtw(path_mfcc1,path_mfcc2):
  X = get_mfcc_features(path=path_mfcc1)
  Y = get_mfcc_features(path=path_mfcc2)
  D, wp = librosa.sequence.dtw(X, Y)
  return D[wp[-1,1],wp[-1,0]]

def get_mfcc_features(path):
    '''
    :param path: files path
    :return: file's features
    '''
    y, sr = librosa.load(path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)

    return stats.zscore(mfcc,axis=1)

def compare_mfcc_features(path_mfcc1,path_mfcc2):
  mfcc1 = get_mfcc_features(path=path_mfcc1)
  mfcc2 = get_mfcc_features(path=path_mfcc2)
  return DTW(mfcc1, mfcc2)

In [ ]:
import pandas as pd

def get_num_of_chunk_as_int(chunk):
  # receives a chunk's name in str and extracts the num as integer
  return (chunk.split(".")[0]).split("-")[1]

def color_cells(val):
    color = 'green' if val == 9 else 'red'
    return f'color: {color}'
params_list = []

df = pd.DataFrame(0,columns = [10,20,30],index =range(100,550,50))


In [ ]:
def file_params_VAD_iterate(aggressive_list=[3],audio_path=None,frame_duration_list=range(10,40,10),padding_duration_list=range(100,550,50)):
  print(f'Filename: {audio_path}')
  for aggressive_it in aggressive_list: #iterate over list
      for frame_duration_it in frame_duration_list: #iterate over list
        column_list = []
        for padding_duration_it in padding_duration_list: #iterate over list
          # Run VAD on said parameters
          print(f'Params: aggr: {aggressive_it} frame_ms: {frame_duration_it} padding: {padding_duration_it}')
          chunks_web = webrtcvad_usage(aggressive=aggressive_it,audio_path=audio_path,frame_duration=frame_duration_it,padding_duration=padding_duration_it,min_chunk_length = 0.85)
          # calculate DTW score between GT and current MFCC features

          directory = '/content/'
          pattern = "chunk-??.wav"
          min_cost_chunk = ""
          current_cost = np.inf
          for chunk in glob.glob(pattern):
            path_chunk = directory + chunk
            path_gt = "/content/mmm_GTT.wav"
            temp = compare_mfcc_features(path_mfcc1=path_chunk, path_mfcc2=path_gt)
            print(f'Comparing GT to {chunk}. score: {temp:.1f}')
            if(temp < current_cost):
              current_cost = temp
              min_cost_chunk = chunk
          df.loc[[padding_duration_it],frame_duration_it] += get_num_of_chunk_as_int(min_cost_chunk) - 9
          # print(df)

      if os.path.isfile('/content/chunk-00.wav'):
        print("last Delete")
        !rm chunk-??.wav
  return df.style.applymap(color_cells)

In [ ]:
def file_params_VAD_iterate_cross_patient_comparison(aggressive_list=[3],audio_path=None,frame_duration_list=range(10,40,10),padding_duration_list=range(100,550,50)):
  print(f'Filename: {audio_path}')
  for aggressive_it in aggressive_list: #iterate over list
      for frame_duration_it in frame_duration_list: #iterate over list
        column_list = []
        for padding_duration_it in padding_duration_list: #iterate over list
          # Run VAD on said parameters
          print(f'Params: aggr: {aggressive_it} frame_ms: {frame_duration_it} padding: {padding_duration_it}')
          chunks_web = webrtcvad_usage(aggressive=aggressive_it,audio_path=audio_path,frame_duration=frame_duration_it,padding_duration=padding_duration_it,min_chunk_length = 0.85)
          # calculate DTW score between GT and current MFCC features

          directory = '/content/'
          pattern = "chunk-??.wav"
          min_cost_chunk = ""
          current_cost = np.inf
          chunk_list = ["chunk-08.wav","chunk-09.wav","chunk-10.wav","chunk-11.wav"]

          for chunk in chunk_list:
            path_chunk = directory + chunk
            if(os.path.isfile(path_chunk) == False ):
              continue
            temp = 0
            debug_df = pd.DataFrame(0,columns = ['127','01','97','151'],index =["scores"])

            for gt_chunk in glob.glob('/content/GT/*.wav'):
              print(f"\nComparing new patient chunk {chunk.split('-')[1].split('.')[0]} to ", gt_chunk.split('/')[3])
              gt_id = gt_chunk.split('/')[3].split("_")[1]
              score = compare_mfcc_features(path_mfcc1=path_chunk, path_mfcc2=gt_chunk)
              # score = libroa_dtw(path_mfcc1=path_chunk, path_mfcc2=gt_chunk)
              debug_df.loc[["scores"],gt_id] = score
              temp += score
            print(debug_df)
            temp = temp/4
            print("average score is ", temp)

            if(temp < current_cost):
              current_cost = temp
              min_cost_chunk = chunk
              print("\nNew min is ", get_num_of_chunk_as_int(min_cost_chunk)," with a score of ", current_cost)
          df.loc[[padding_duration_it],frame_duration_it] += int(get_num_of_chunk_as_int(min_cost_chunk))
          # print(df)

      if os.path.isfile('/content/chunk-00.wav'):
        print("last Delete")
        !rm chunk-??.wav
  return df.style.applymap(color_cells)

In [ ]:
#                       F - Scar                             M - Scar                  M - Healthy                        F - Healthy
audio_paths_list = ['/content/pat_127_sox_mono.wav','/content/pat_01_sox_mono.wav','/content/pat_97_sox_mono.wav','/content/pat_151_sox_mono.wav']

In [ ]:
# Get the ground truth for chunk #9 - the "mmm" audio from patient 127
if (not os.path.isdir("/content/GT")):
  os.mkdir("/content/GT")


chunks_web = webrtcvad_usage(aggressive=3,audio_path=audio_paths_list[0],frame_duration=10,padding_duration=100,min_chunk_length=0.8)
os.rename('/content/chunk-09.wav', "/content/GT/pat_127_GT.wav")


chunks_web = webrtcvad_usage(aggressive=3,audio_path=audio_paths_list[1],frame_duration=10,padding_duration=100,min_chunk_length=0.8)
os.rename('/content/chunk-09.wav', "/content/GT/pat_01_GT.wav")


chunks_web = webrtcvad_usage(aggressive=3,audio_path=audio_paths_list[2],frame_duration=10,padding_duration=100,min_chunk_length=0.8)
os.rename('/content/chunk-09.wav', "/content/GT/pat_97_GT.wav")


chunks_web = webrtcvad_usage(aggressive=3,audio_path=audio_paths_list[3],frame_duration=10,padding_duration=100,min_chunk_length=0.8)
os.rename('/content/chunk-09.wav', "/content/GT/pat_151_GT.wav")

!rm chunk-??.wav

In [ ]:
pat_num ='74'

!sox /content/drive/MyDrive/Study_materials/Voice\ samples\ study/patient\ {pat_num}.wav -c 1 pat_{pat_num}_sox.wav
!sox /content/drive/MyDrive/Study_materials/Voice\ samples\ study/Patient\ {pat_num}.wav -c 1 pat_{pat_num}_sox.wav
!sox /content/pat_{pat_num}_sox.wav -r 16000 /content/pat_{pat_num}_sox_mono.wav
!rm pat_{pat_num}_sox.wav


In [ ]:
a = {}
df = pd.DataFrame(0,columns = [10,20],index =range(100,350,50))
path = '/content/pat_'+pat_num+'_sox_mono.wav'
num = path.split("_")[1]
a[num] = file_params_VAD_iterate_cross_patient_comparison(audio_path=path,frame_duration_list=[10,20],padding_duration_list=range(100,350,50))


In [ ]:
a['74']

In [ ]:
for path in audio_paths_list:
  # a = file_params_VAD_iterate(audio_path=path,frame_duration_list=range(20,40,10),padding_duration_list=range(400,600,100))
  df = pd.DataFrame(0,columns = [10,20,30],index =range(100,550,50))
  num = path.split("_")[1]
  a[num] = file_params_VAD_iterate(audio_path=path)

In [ ]:
chunks_web = webrtcvad_usage(aggressive=3,audio_path='/content/pat_'+pat_num+'_sox_mono.wav',frame_duration=20,padding_duration=150,min_chunk_length = 0.85)

In [ ]:
df_present = pd.DataFrame(0,columns = ["01","127","151","97"],index =range(0,18,1))

directory = '/content/'
pattern = "chunk-??.wav"
# chunk_list = ["chunk-08.wav","chunk-09.wav","chunk-10.wav","chunk-11.wav"]

for chunk in glob.glob('/content/chunk-??.wav'):
  print(chunk)
  path_chunk = chunk
  if(os.path.isfile(path_chunk) == False ):
    continue

  for gt_chunk in glob.glob('/content/GT/*.wav'):
    print(f"\nComparing new patient chunk {get_num_of_chunk_as_int(chunk)} to ", gt_chunk.split('/')[3])
    gt_id = gt_chunk.split('/')[3].split("_")[1]
    score = compare_mfcc_features(path_mfcc1=path_chunk, path_mfcc2=gt_chunk)
    df_present.loc[[int(get_num_of_chunk_as_int(chunk))],gt_id] = score
  print(df_present)

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format
df_present
temps = pd.DataFrame(1,columns = ["avg"],index =range(0,18,1))
f = df_present.div(df_present.sum(axis=0), axis=1)
temps = f.mean(axis=1)
print(temps)
# res = pd.concat([f,temps],axis=1)
# res

In [ ]:
a["74"]

In [ ]:
dftest = pd.DataFrame(0,columns = ["01","127","151","97"],index =range(0,18,1))
dftest2 = pd.DataFrame(0,columns = ["01","127","151","97"],index =range(0,18,1))
dftest

In [ ]:
directory = '/content/'
pattern = "chunk-??.wav"

for new_pat_chunk in glob.glob('/content/chunk-??.wav'):
  new_pat_num = int(get_num_of_chunk_as_int(new_pat_chunk))
  print(f'Comparing {new_pat_num} ',end ="")
  for gt_chunk in glob.glob('/content/GT/*.wav'):
    gt_num = gt_chunk.split("_")[1]
    print(f'{gt_num}, ',end ="")
    score = libroa_dtw(path_mfcc1=new_pat_chunk, path_mfcc2=gt_chunk)
    mfccscore = compare_mfcc_features(path_mfcc1=new_pat_chunk, path_mfcc2=gt_chunk)
    dftest.loc[[new_pat_num],gt_num] = score
    dftest2.loc[[new_pat_num],gt_num] = mfccscore
  print("\n")

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
dftest2

In [ ]:
waveform, sample_rate = torchaudio.load('chunk-09.wav')
Audio(waveform.numpy()[0], rate=16000)

# audio_labels = ["0 normal pitch E","1 low pitch E","2 high pitch E",
                # "3 normal pitch Ah","4 low pitch Ah","5 high pitch Ah",
                # "6 normal pitch Ooh","7 low pitch Ooh","8 high pitch Ooh",
                # "9 normal pitch Mmmmm"]

In [ ]:
waveform2, sample_rate = torchaudio.load('/content/GT/pat_97_GT.wav')
Audio(waveform2.numpy()[0], rate=16000)


In [ ]:
waveform4, sample_rate = torchaudio.load('chunk-11.wav')
Audio(waveform4.numpy()[0], rate=16000)

In [ ]:
def plot_specgram(waveform, sample_rate, title="Spectrogram"):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
    figure.suptitle(title)
    plt.show(block=False)

In [ ]:
Audio(waveform2.numpy()[0],rate=16000)

In [ ]:
import speech_recognition as sr

filename = "/content/chunk-09.wav"

# initialize the recognizer
r = sr.Recognizer()

# open the file
with sr.AudioFile(filename) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data)
    print(text)
